### Part 3: Workflow ของการวน loop เลือกตัวแปรเพื่อนำมาสร้างสมการให้ครบทุกรูปแบบที่เป็นไปได้
ประกอบด้วย 3 loops <br>
Loop 1: เปลี่ยนจำนวน feature ที่ใช้ <br>
Loop 2: เปลี่ยนกลุ่ม feature ที่จะนำมาใช้สร้างสมการ regression <br>
Loop 3: เปลี่ยน polynomial degree 

<center><h3>Workflow Image</h3></center>
<img src="workflow.png" style="width: 500px;" class="center">

In [1]:
# Import Libraries
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import itertools
from decimal import Decimal

In [2]:
# ดึงข้อมูลจากไฟล์ csv
df = pd.read_csv("input.csv")
df.head()

,Y,X1,X2,X3
0,222000.0,1,6.3,2016
1,177000.0,1,5.5,2016
2,177777.0,1,5.5,2016
3,199999.0,1,5.5,2016
4,199999.0,1,5.5,2016


In [3]:
# แบ่ง dataframe เป็นตัวแปร X และ y
df_X = df.drop(df.columns[[0]], axis=1) #X input
y = df[df.columns[0]]

In [4]:
# กำหนดค่าตัวแปร
deg = 3 # specify maximum polynomial degree
n = df_X.shape[1] # get number of feature

<b>Loop 1: เปลี่ยนจำนวน feature ที่ใช้</b>
- จัดกลุ่ม feature ให้ครบทุกรูปแบบที่เป็นไปได้ ตามจำนวน feature ที่ใช้

Note: จำนวนรูปแบบเป็นไปตามสูตร Combination (nCr)
    
ใช้ function ชื่อ itertools.combinations ในการจัดกลุ่ม feature<br>
df_X.columns คือ list ของ feature ทั้งหมด

In [5]:
for num_feature in range(1,n+1):
    print("\n start running number of feature:"+str(num_feature))
    feature_list = list(itertools.combinations(df_X.columns,num_feature)) # get all combination of features
    print(feature_list)
    feature = [None] * num_feature # create empty array "feature"


 start running number of feature:1
[('X1',), ('X2',), ('X3',)]

 start running number of feature:2
[('X1', 'X2'), ('X1', 'X3'), ('X2', 'X3')]

 start running number of feature:3
[('X1', 'X2', 'X3')]


<b>Loop 2: เปลี่ยนกลุ่ม feature ที่จะนำมาใช้สร้างสมการ regression</b>
- เลือกกลุ่ม feature ที่ใช้ จากกลุ่มทั้งหมดที่ได้จาก Loop 1
- สร้าง dataframe ใหม่จาก feature ที่ใช้

In [6]:
feature_list = [('X1', 'X2'), ('X1', 'X3'), ('X2', 'X3')] # กลุ่มของ feature ทั้งหมดที่เป็นไปได้ (กรณีเลือกใช้ 2 feature)

for feature in feature_list:
    print("\n features:"+str(feature))
    X = df_X[list(feature)] # create new dataframe from selected feature(s)


 features:('X1', 'X2')

 features:('X1', 'X3')

 features:('X2', 'X3')


<b> Loop 3: เปลี่ยน polynomial degree</b>
- เพิ่ม degree จาก 1 ไปถึง 3
- fit สมการ

In [7]:
# สร้าง function สำหรับ print สมการ (อ่านข้ามส่วนนี้ไปได้)
# Function: Generate polynomial equation
def Poly_equation(feature, coef, intercept, power):
    '''
    Return fitted polynomial equation as a string
    
    feature: list of feature in dataset
    power: degree of each feature in each term (can get from poly.powers_)
    '''
    poly_string = ""
    
    for i in range(len(coef)): # create polynomial term
        
        #Coefficients
        if i == 0:
            term_string = "y = %.2E" % Decimal(coef[i])
        elif coef[i] >= 0: # add + sign in front of coef
            term_string = "+%.2E" % Decimal(coef[i])
        else:
            term_string = "%.2E" % Decimal(coef[i])
        
        #Powers
        feature_order = 0
        for power_iter in power[i]: # power for each feature
            if power_iter == 1 : #degree of that feature = 1
                term_string += '*' + str(feature[feature_order])
            elif power_iter > 1 : #degree of that feature > 1
                term_string += '*' + str(feature[feature_order]) + '^' + str(power_iter)
            feature_order += 1
        poly_string += term_string
    
    #Add intercept
    if intercept >= 0:
        poly_string += "+"
    poly_string += "%.2E" % Decimal(intercept)
    
    return poly_string

In [8]:
feature = ['X1','X2']
X = df_X[list(feature)]
for nPoly in range(1, deg+1):
    poly = PolynomialFeatures(degree=nPoly, include_bias=None)
    # transform features to polynomial terms
    X_ = poly.fit_transform(X)  
        
    # train linear regression model
    lg = LinearRegression()
    lg.fit(X_, y)
     
    # ใน Loop สุดท้าย สามารถ แสดงสมการ, กราฟ และค่า R-Square ได้
    print("Polynomial degree: "+str(nPoly))
    print("Degree for each feature:\n" + str(poly.powers_))
    print("Equation: " + Poly_equation(feature, lg.coef_, lg.intercept_, poly.powers_))
    print("R-Square = "+ str(np.round(lg.score(X_,y),3))+"\n")

Polynomial degree: 1
Degree for each feature:
[[1 0]
 [0 1]]
Equation: y = -1.15E+02*X1+1.15E+04*X2+1.00E+04
R-Square = 0.411

Polynomial degree: 2
Degree for each feature:
[[1 0]
 [0 1]
 [2 0]
 [1 1]
 [0 2]]
Equation: y = -5.00E+01*X1+2.87E+04*X2+3.85E-01*X1^2-8.82E+01*X1*X2-8.58E+02*X2^2-1.62E+04
R-Square = 0.586

Polynomial degree: 3
Degree for each feature:
[[1 0]
 [0 1]
 [2 0]
 [1 1]
 [0 2]
 [3 0]
 [2 1]
 [1 2]
 [0 3]]
Equation: y = 6.78E+01*X1+2.39E+04*X2+1.72E-01*X1^2-1.63E+02*X1*X2+2.06E+03*X2^2-6.79E-04*X1^3+3.06E-01*X1^2*X2-5.72E+00*X1*X2^2-2.14E+02*X2^3-1.61E+04
R-Square = 0.639

